<a href="https://colab.research.google.com/github/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/blob/main/Opt_In_VariacionEn_Demanda_y_LeadTima.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# URL del archivo Excel
url = "https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx"

# Leer datos
ventas_df = pd.read_excel(url, sheet_name="Historico")
lead_time_df = pd.read_excel(url, sheet_name="LeadTime-Dias")

# Limpiar nombres de ITEM
ventas_df['ITEM'] = ventas_df['ITEM'].str.strip()
lead_time_df['ITEM'] = lead_time_df['ITEM'].str.strip()

# Lista para resultados
resultados = []

# Parámetros
Z = 1.645  # Nivel de servicio 95%
R_dias = 0.5  # Tiempo de reorden en días
MIN_DATOS_PARA_CALCULO = 18  # Usamos los últimos 18 meses

for idx, row in ventas_df.iterrows():
    item = row['ITEM']

    # Extraer ventas (columnas de mes)
    ventas = pd.to_numeric(row[1:], errors='coerce')

    # --- 🔧 Imputar valores negativos ---
    ventas_imp = ventas.copy()
    negativos = ventas_imp < 0

    for i in negativos[negativos].index:
        try:
            pos = list(ventas.index).index(i)
        except:
            continue
        inicio_ventana = max(0, pos - 12)
        ventana = ventas.iloc[inicio_ventana:pos]
        validos = ventana[(ventana > 0) & (ventana.notna())]

        if len(validos) >= 3:
            estimado = validos.median()
        elif len(validos) > 0:
            estimado = validos.mean()
        else:
            todos_positivos = ventas[(ventas > 0) & (ventas.notna())]
            estimado = todos_positivos.mean() if len(todos_positivos) > 0 else 0

        ventas_imp[i] = max(0, estimado)

    ventas_imp = ventas_imp.clip(lower=0)

    # --- 📊 Últimos 18 meses para demanda ---
    ultimos_18 = ventas_imp[-18:]

    if len(ultimos_18.dropna()) < MIN_DATOS_PARA_CALCULO:
        resultados.append({
            'ITEM': item,
            'Stock_Seguridad_Dias': np.nan,
            'SS_Techo_Dias': np.nan,
            'SS_Optimo_Dias': np.nan
        })
        continue

    # Promedio y desviación de demanda (en unidades/mes)
    mu_d = ultimos_18.mean()
    sigma_d = ultimos_18.std()

    # Si no hay variabilidad, asignar un mínimo
    if pd.isna(sigma_d) or sigma_d == 0:
        sigma_d = 0.1 * mu_d if mu_d > 0 else 0.1

    # --- Lead Time en días ---
    if item in lead_time_df['ITEM'].values:
        lt_row = lead_time_df[lead_time_df['ITEM'] == item].iloc[0, 1:]
        lt_values = pd.to_numeric(lt_row, errors='coerce').dropna()
        if len(lt_values) > 0:
            mu_L = lt_values.mean()  # Promedio de lead time en días
            sigma_L = lt_values.std()  # Desviación de lead time en días
        else:
            mu_L = 5.0
            sigma_L = 0.5
    else:
        print(f"Advertencia: Sin lead time para {item}. Usando L = 5 días, σL = 0.5.")
        mu_L = 5.0
        sigma_L = 0.5

    # --- Fórmula completa de Stock de Seguridad en UNIDADES ---
    # Ss = Z * sqrt((μ_L + R)*σ_d² + σ_L²*μ_d²)
    try:
        SS_unidades = Z * np.sqrt(
            (mu_L + R_dias) * sigma_d**2 +
            sigma_L**2 * mu_d**2
        )
    except:
        SS_unidades = 0.1

    # Convertir SS a días
    if mu_d > 0:
        SS_dias = (SS_unidades / mu_d) * 30  # Ajuste a días (por mes de 30 días)
        SS_dias = round(SS_dias, 2)
    else:
        SS_dias = 0.0

    # --- Stock de Seguridad Techo en Días ---
    SS_techo_dias = SS_dias + mu_L + R_dias
    SS_techo_dias = round(SS_techo_dias, 2)

    # --- Stock Óptimo en Días ---
    SS_optimo_dias = SS_dias + (mu_L + R_dias) / 2
    SS_optimo_dias = round(SS_optimo_dias, 2)

    # Agregar al resultado
    resultados.append({
        'ITEM': item,
        'Stock_Seguridad_Dias': SS_dias,
        'SS_Techo_Dias': SS_techo_dias,
        'SS_Optimo_Dias': SS_optimo_dias
    })

# Crear DataFrame final
resultado_df = pd.DataFrame(resultados)

print("\n✅ Resultados con fórmula mejorada (variabilidad en demanda y lead time):")
print(resultado_df)

# Opcional: guardar
# resultado_df.to_excel("Stock_Seguridad_Final_Mejorado.xlsx", index=False)

In [1]:
import pandas as pd
import numpy as np

# URL del archivo Excel
url = "https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx"

# Leer datos
ventas_df = pd.read_excel(url, sheet_name="Historico")
lead_time_df = pd.read_excel(url, sheet_name="LeadTime-Dias")

# Limpiar nombres
ventas_df['ITEM'] = ventas_df['ITEM'].str.strip()
lead_time_df['ITEM'] = lead_time_df['ITEM'].str.strip()

# Lista para resultados
resultados = []

# Parámetros
Z = 1.645  # 95% nivel de servicio
R_dias = 0.5  # Tiempo de reorden en días
R_meses = R_dias / 30  # Convertir a meses
MIN_DATOS_PARA_CALCULO = 18

for idx, row in ventas_df.iterrows():
    item = row['ITEM']

    # --- 🔹 Extraer y limpiar demanda ---
    ventas = pd.to_numeric(row[1:], errors='coerce')

    # Imputar negativos
    ventas_imp = ventas.copy()
    negativos = ventas_imp < 0

    for i in negativos[negativos].index:
        try:
            pos = list(ventas.index).index(i)
        except:
            continue
        inicio_ventana = max(0, pos - 12)
        ventana = ventas.iloc[inicio_ventana:pos]
        validos = ventana[(ventana > 0) & (ventana.notna())]

        if len(validos) >= 3:
            estimado = validos.median()
        elif len(validos) > 0:
            estimado = validos.mean()
        else:
            todos_positivos = ventas[(ventas > 0) & (ventas.notna())]
            estimado = todos_positivos.mean() if len(todos_positivos) > 0 else 0

        ventas_imp[i] = max(0, estimado)

    ventas_imp = ventas_imp.clip(lower=0)

    # Últimos 18 meses
    ultimos_18 = ventas_imp[-18:]
    if len(ultimos_18.dropna()) < MIN_DATOS_PARA_CALCULO:
        resultados.append({
            'ITEM': item,
            'Stock_Seguridad_Dias': np.nan,
            'SS_Techo_Dias': np.nan,
            'SS_Optimo_Dias': np.nan
        })
        continue

    mu_d = ultimos_18.mean()        # Demanda promedio mensual
    sigma_d = ultimos_18.std()      # Desviación demanda mensual

    if pd.isna(sigma_d) or sigma_d == 0:
        sigma_d = 0.1 * mu_d if mu_d > 0 else 0.1

    # --- 🔹 Lead Time: en DÍAS (convertir a meses dividiendo para 30) ---
    if item in lead_time_df['ITEM'].values:
        lt_row = lead_time_df[lead_time_df['ITEM'] == item].iloc[0, 1:]
        lt_vals_dias = pd.to_numeric(lt_row, errors='coerce').dropna()
        if len(lt_vals_dias) > 0:
            mu_L_dias = lt_vals_dias.mean()      # Promedio en días
            sigma_L_dias = lt_vals_dias.std()    # Desviación en días
            mu_L = mu_L_dias / 30                # Convertir a meses
            sigma_L = sigma_L_dias / 30          # Desviación en meses
        else:
            mu_L = 5 / 30
            sigma_L = 1 / 30  # desviación mínima razonable
    else:
        print(f"Advertencia: Sin lead time para {item}. Usando 5 días.")
        mu_L = 5 / 30
        sigma_L = 1 / 30

    if pd.isna(sigma_L) or sigma_L == 0:
        sigma_L = 0.1 * mu_L if mu_L > 0 else 0.1 / 30

    # --- 🔹 Fórmula completa del Stock de Seguridad (en unidades) ---
    try:
        SS_unidades = Z * np.sqrt(
            (mu_L + R_meses) * sigma_d**2 +   # variabilidad demanda
            (sigma_L**2) * mu_d**2            # variabilidad lead time
        )
    except Exception as e:
        SS_unidades = 0.1

    # --- 🔹 Convertir SS a DÍAS ---
    if mu_d > 0:
        SS_dias = (SS_unidades / mu_d) * 30  # porque mu_d es mensual → (unidades / unidades/mes) = meses → *30 = días
        SS_dias = round(SS_dias, 2)
    else:
        SS_dias = 0.0

    # --- 🔹 Stock de Seguridad Techo (en días) ---
    L_dias = mu_L * 30  # volver a días
    SS_techo_dias = SS_dias + L_dias + R_dias
    SS_techo_dias = round(SS_techo_dias, 2)

    # --- 🔹 Stock Óptimo en Días ---
    SS_optimo_dias = SS_dias + (L_dias + R_dias) / 2
    SS_optimo_dias = round(SS_optimo_dias, 2)

    # Guardar resultados
    resultados.append({
        'ITEM': item,
        'Stock_Seguridad_Dias': SS_dias,
        'SS_Techo_Dias': SS_techo_dias,
        'SS_Optimo_Dias': SS_optimo_dias
    })

# Crear DataFrame final
resultado_df = pd.DataFrame(resultados)

print("\n✅ Resultados finales (Lead Time en días → convertido a meses dividiendo entre 30):")
print(resultado_df)

# Opcional: exportar
# resultado_df.to_excel("Stock_Seguridad_Final_Final.xlsx", index=False)

/tmp/ipython-input-3911092054.py:51: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '24132.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  ventas_imp[i] = max(0, estimado)



✅ Resultados finales (Lead Time en días → convertido a meses dividiendo entre 30):
         ITEM  Stock_Seguridad_Dias  SS_Techo_Dias  SS_Optimo_Dias
0      ITEM 1                  4.65           9.62            7.13
1      ITEM 2                  8.50          21.50           15.00
2      ITEM 3                  3.16           7.03            5.09
3      ITEM 4                  6.47          19.92           13.20
4      ITEM 5                  6.17          17.66           11.91
..        ...                   ...            ...             ...
655  ITEM 656                  4.74          11.64            8.19
656  ITEM 657                  4.62          12.30            8.46
657  ITEM 658                  5.49          14.84           10.17
658  ITEM 659                  5.73          16.32           11.02
659  ITEM 660                  5.37          14.86           10.11

[660 rows x 4 columns]


In [2]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
from scipy.stats import norm

# URL del archivo Excel
url = "https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx"

# Descargar archivo
response = requests.get(url)
excel_data = BytesIO(response.content)

# Leer hojas
historico_df = pd.read_excel(excel_data, sheet_name='Historico')
forecast_df = pd.read_excel(excel_data, sheet_name='Forecast')
precios_df = pd.read_excel(excel_data, sheet_name='Precios-Costos')
lead_time_df = pd.read_excel(excel_data, sheet_name='LeadTime-Dias')

print("="*80)
print("🚀 INICIANDO ANÁLISIS DE INVENTARIOS - ABC-XYZ + STOCK DE SEGURIDAD MEJORADO")
print("="*80)

# ============================================================================ #
# PARTE 1: CLASIFICACIÓN ABC
# ============================================================================ #
print("\n📊 Calculando Clasificación ABC...")

forecast_df['ITEM'] = forecast_df['ITEM'].astype(str).str.strip()
precios_df['ITEM'] = precios_df['ITEM'].astype(str).str.strip()
forecast_df.columns = forecast_df.columns.str.strip()

# Ventas anuales
monthly_cols = [col for col in forecast_df.columns if col != 'ITEM']
forecast_df['Total_Ventas_Anual'] = forecast_df[monthly_cols].sum(axis=1)

# Limpiar precios
precios_df['Precio'] = precios_df['Precio'].astype(str).str.replace(r'[$,]', '', regex=True).astype(float)
precios_df['Costo'] = precios_df['Costo'].astype(str).str.replace(r'[$,]', '', regex=True).astype(float)

# Unir y calcular valor
abc_df = forecast_df[['ITEM', 'Total_Ventas_Anual']].merge(
    precios_df[['ITEM', 'Precio']], on='ITEM', how='inner')
abc_df['Valor_Total_Dolares'] = abc_df['Total_Ventas_Anual'] * abc_df['Precio']

# Ordenar y acumular
abc_df = abc_df.sort_values('Valor_Total_Dolares', ascending=False)
abc_df['Porcentaje_Acumulado'] = abc_df['Valor_Total_Dolares'].cumsum() / abc_df['Valor_Total_Dolares'].sum() * 100

# Clasificar ABC
abc_df['Clase_ABC'] = abc_df['Porcentaje_Acumulado'].apply(
    lambda x: 'A' if x <= 60 else 'B' if x <= 80 else 'C')

print(f"   ✅ Items clasificados ABC: {len(abc_df)}")

# ============================================================================ #
# PARTE 2: CLASIFICACIÓN XYZ
# ============================================================================ #
print("\n📈 Calculando Clasificación XYZ...")

# Últimos 18 meses
xyz_data = historico_df.set_index('ITEM').iloc[:, -18:].copy()

# Limpieza de datos
def clean_xyz_row(row):
    row = pd.to_numeric(row, errors='coerce')
    neg_or_zero = (row <= 0) | (row.isna())
    if neg_or_zero.any():
        valid = row[(row > 0) & (row.notna())]
        fill_value = valid.median() if not valid.empty else 0
        row[neg_or_zero] = fill_value
    return row.clip(lower=0)

xyz_clean = xyz_data.apply(clean_xyz_row, axis=1)

# CV
mean = xyz_clean.mean(axis=1)
std = xyz_clean.std(axis=1)
cv = std / mean
cv = cv.replace([np.inf, -np.inf], 1000).fillna(1000)

# Cuantiles para XYZ
p33 = cv.quantile(0.33)
p67 = cv.quantile(0.67)

xyz_df = pd.DataFrame({'ITEM': xyz_clean.index, 'CV': cv.values})
xyz_df['Clase_XYZ'] = pd.cut(cv, bins=[-1, p33, p67, np.inf], labels=['X', 'Y', 'Z'])
xyz_df['ITEM'] = xyz_df['ITEM'].astype(str).str.strip()

print(f"   ✅ Items clasificados XYZ: {len(xyz_df)}")

# ============================================================================ #
# PARTE 3: COMBINAR ABC-XYZ
# ============================================================================ #
print("\n🔄 Combinando clasificaciones ABC-XYZ...")

abc_minimal = abc_df[['ITEM', 'Clase_ABC']].copy()
abc_minimal['ITEM'] = abc_minimal['ITEM'].astype(str).str.strip()

clasificacion_df = abc_minimal.merge(xyz_df[['ITEM', 'Clase_XYZ']], on='ITEM', how='inner')
clasificacion_df['Clasificacion_ABC_XYZ'] = clasificacion_df['Clase_ABC'] + clasificacion_df['Clase_XYZ']

print(f"   ✅ Items con clasificación completa: {len(clasificacion_df)}")

# ============================================================================ #
# PARTE 4: NIVELES DE SERVICIO Y Z-SCORE
# ============================================================================ #
print("\n⚙️ Asignando niveles de servicio por clase...")

niveles_servicio = {
    'AX': 0.95, 'AY': 0.90, 'AZ': 0.85,
    'BX': 0.85, 'BY': 0.80, 'BZ': 0.75,
    'CX': 0.75, 'CY': 0.70, 'CZ': 0.65
}

clasificacion_df['Nivel_Servicio'] = clasificacion_df['Clasificacion_ABC_XYZ'].map(niveles_servicio)
clasificacion_df['Z_Score'] = clasificacion_df['Nivel_Servicio'].apply(
    lambda x: norm.ppf(x) if pd.notna(x) else 1.645  # 95% si no hay clase
)

print("   📋 Niveles de servicio asignados:")
for clase, nivel in niveles_servicio.items():
    count = len(clasificacion_df[clasificacion_df['Clasificacion_ABC_XYZ'] == clase])
    z = norm.ppf(nivel)
    print(f"      {clase}: {nivel*100:.0f}% (Z={z:.3f}) - {count} items")

# ============================================================================ #
# PARTE 5: STOCK DE SEGURIDAD CON FÓRMULA COMPLETA
# ============================================================================ #
print("\n🛡️ Calculando Stock de Seguridad (modelo completo)...")

historico_df['ITEM'] = historico_df['ITEM'].str.strip()
lead_time_df['ITEM'] = lead_time_df['ITEM'].str.strip()

R_dias = 0.5
R_meses = R_dias / 30
MIN_DATOS = 18

resultados_ss = []

for idx, row in clasificacion_df.iterrows():
    item = row['ITEM']
    Z = row['Z_Score']

    # Buscar datos históricos
    item_hist = historico_df[historico_df['ITEM'] == item]
    if len(item_hist) == 0:
        resultados_ss.append({
            'ITEM': item, 'Stock_Seguridad_Dias': np.nan,
            'Promedio_Ventas_Mensual': np.nan, 'Desviacion_Ventas': np.nan,
            'Lead_Time_Promedio_Dias': np.nan, 'Lead_Time_Desv_Dias': np.nan
        })
        continue

    # Extraer ventas
    ventas = pd.to_numeric(item_hist.iloc[0, 1:], errors='coerce')

    # Imputar negativos
    ventas_imp = ventas.copy()
    negativos = ventas_imp < 0
    for i in negativos[negativos].index:
        pos = list(ventas.index).index(i) if i in ventas.index else None
        if pos is None: continue
        inicio = max(0, pos - 12)
        ventana = ventas.iloc[inicio:pos]
        validos = ventana[(ventana > 0) & (ventana.notna())]
        if len(validos) >= 3:
            estimado = validos.median()
        elif len(validos) > 0:
            estimado = validos.mean()
        else:
            todos = ventas[(ventas > 0) & (ventas.notna())]
            estimado = todos.mean() if len(todos) > 0 else 0
        ventas_imp[i] = max(0, estimado)
    ventas_imp = ventas_imp.clip(lower=0)

    # Últimos 18 meses
    ultimos_18 = ventas_imp[-18:]
    if len(ultimos_18.dropna()) < MIN_DATOS:
        resultados_ss.append({
            'ITEM': item, 'Stock_Seguridad_Dias': np.nan,
            'Promedio_Ventas_Mensual': np.nan, 'Desviacion_Ventas': np.nan,
            'Lead_Time_Promedio_Dias': np.nan, 'Lead_Time_Desv_Dias': np.nan
        })
        continue

    mu_d = ultimos_18.mean()
    sigma_d = ultimos_18.std()
    if pd.isna(sigma_d) or sigma_d == 0:
        sigma_d = 0.1 * mu_d if mu_d > 0 else 0.1

    # Lead Time en DÍAS → convertir a meses
    lt_item = lead_time_df[lead_time_df['ITEM'] == item]
    if len(lt_item) > 0:
        lt_vals = pd.to_numeric(lt_item.iloc[0, 1:], errors='coerce').dropna()
        if len(lt_vals) > 0:
            mu_L_dias = lt_vals.mean()
            sigma_L_dias = lt_vals.std()
        else:
            mu_L_dias, sigma_L_dias = 5.0, 1.0
    else:
        mu_L_dias, sigma_L_dias = 5.0, 1.0

    # Convertir a meses
    mu_L = mu_L_dias / 30
    sigma_L = sigma_L_dias / 30

    # Fórmula completa en unidades
    try:
        SS_unidades = Z * np.sqrt(
            (mu_L + R_meses) * sigma_d**2 +
            (sigma_L**2) * mu_d**2
        )
    except:
        SS_unidades = 0.1

    # Convertir a días
    if mu_d > 0:
        SS_dias = (SS_unidades / mu_d) * 30
        SS_dias = round(SS_dias, 2)
    else:
        SS_dias = 0.0

    resultados_ss.append({
        'ITEM': item,
        'Stock_Seguridad_Dias': SS_dias,
        'Promedio_Ventas_Mensual': round(mu_d, 2),
        'Desviacion_Ventas': round(sigma_d, 2),
        'Lead_Time_Promedio_Dias': round(mu_L_dias, 1),
        'Lead_Time_Desv_Dias': round(sigma_L_dias, 1)
    })

print(f"   ✅ Stock de seguridad calculado para {len(resultados_ss)} items")

# ============================================================================ #
# PARTE 6: CONSOLIDAR Y EXPORTAR
# ============================================================================ #
print("\n📋 Consolidando resultados...")

ss_df = pd.DataFrame(resultados_ss)
resultado_final = clasificacion_df.merge(ss_df, on='ITEM', how='left')

# Unir con datos de valor
resultado_final = resultado_final.merge(
    abc_df[['ITEM', 'Total_Ventas_Anual', 'Precio', 'Valor_Total_Dolares']],
    on='ITEM', how='left'
)

# Reordenar columnas
columnas = [
    'ITEM', 'Clasificacion_ABC_XYZ', 'Clase_ABC', 'Clase_XYZ',
    'Nivel_Servicio', 'Z_Score', 'Stock_Seguridad_Dias',
    'Total_Ventas_Anual', 'Precio', 'Valor_Total_Dolares',
    'Promedio_Ventas_Mensual', 'Desviacion_Ventas',
    'Lead_Time_Promedio_Dias', 'Lead_Time_Desv_Dias'
]

resultado_final = resultado_final[columnas]

# Exportar
resultado_final.to_excel("Analisis_ABC_XYZ_Stock_Seguridad_Completo.xlsx", index=False)

print(f"\n✅ Exportado a: Analisis_ABC_XYZ_Stock_Seguridad_Completo.xlsx")
print(f"\n🎯 Total de ítems: {len(resultado_final)}")

print("\n📊 Stock de Seguridad Promedio por Clase:")
promedio_ss = resultado_final.groupby('Clasificacion_ABC_XYZ')['Stock_Seguridad_Dias'].agg(['mean', 'count']).round(1)
for clase, row in promedio_ss.iterrows():
    print(f"   {clase}: {row['mean']:5.1f} días (n={int(row['count'])})")

print("\n📋 Ejemplo de resultados:")
print(resultado_final[['ITEM', 'Clasificacion_ABC_XYZ', 'Nivel_Servicio', 'Stock_Seguridad_Dias']].head(10).to_string(index=False))

print("="*80)

🚀 INICIANDO ANÁLISIS DE INVENTARIOS - ABC-XYZ + STOCK DE SEGURIDAD MEJORADO

📊 Calculando Clasificación ABC...
   ✅ Items clasificados ABC: 660

📈 Calculando Clasificación XYZ...
   ✅ Items clasificados XYZ: 660

🔄 Combinando clasificaciones ABC-XYZ...


TypeError: Object with dtype category cannot perform the numpy op add

In [3]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
from scipy.stats import norm

# URL del archivo Excel
url = "https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx"

# Descargar archivo
response = requests.get(url)
excel_data = BytesIO(response.content)

# Leer hojas
historico_df = pd.read_excel(excel_data, sheet_name='Historico')
forecast_df = pd.read_excel(excel_data, sheet_name='Forecast')
precios_df = pd.read_excel(excel_data, sheet_name='Precios-Costos')
lead_time_df = pd.read_excel(excel_data, sheet_name='LeadTime-Dias')

print("="*80)
print("🚀 INICIANDO ANÁLISIS DE INVENTARIOS - ABC-XYZ + STOCK DE SEGURIDAD MEJORADO")
print("="*80)

# ============================================================================ #
# PARTE 1: CLASIFICACIÓN ABC
# ============================================================================ #
print("\n📊 Calculando Clasificación ABC...")

forecast_df['ITEM'] = forecast_df['ITEM'].astype(str).str.strip()
precios_df['ITEM'] = precios_df['ITEM'].astype(str).str.strip()
forecast_df.columns = forecast_df.columns.str.strip()

# Ventas anuales
monthly_cols = [col for col in forecast_df.columns if col != 'ITEM']
forecast_df['Total_Ventas_Anual'] = forecast_df[monthly_cols].sum(axis=1)

# Limpiar precios
precios_df['Precio'] = precios_df['Precio'].astype(str).str.replace(r'[$,]', '', regex=True).astype(float)
precios_df['Costo'] = precios_df['Costo'].astype(str).str.replace(r'[$,]', '', regex=True).astype(float)

# Unir y calcular valor
abc_df = forecast_df[['ITEM', 'Total_Ventas_Anual']].merge(
    precios_df[['ITEM', 'Precio']], on='ITEM', how='inner')
abc_df['Valor_Total_Dolares'] = abc_df['Total_Ventas_Anual'] * abc_df['Precio']

# Ordenar y acumular
abc_df = abc_df.sort_values('Valor_Total_Dolares', ascending=False)
abc_df['Porcentaje_Acumulado'] = abc_df['Valor_Total_Dolares'].cumsum() / abc_df['Valor_Total_Dolares'].sum() * 100

# Clasificar ABC
abc_df['Clase_ABC'] = abc_df['Porcentaje_Acumulado'].apply(
    lambda x: 'A' if x <= 60 else 'B' if x <= 80 else 'C')

print(f"   ✅ Items clasificados ABC: {len(abc_df)}")

# ============================================================================ #
# PARTE 2: CLASIFICACIÓN XYZ
# ============================================================================ #
print("\n📈 Calculando Clasificación XYZ...")

# Últimos 18 meses
xyz_data = historico_df.set_index('ITEM').iloc[:, -18:].copy()

# Limpieza de datos
def clean_xyz_row(row):
    row = pd.to_numeric(row, errors='coerce')
    neg_or_zero = (row <= 0) | (row.isna())
    if neg_or_zero.any():
        valid = row[(row > 0) & (row.notna())]
        fill_value = valid.median() if not valid.empty else 0
        row[neg_or_zero] = fill_value
    return row.clip(lower=0)

xyz_clean = xyz_data.apply(clean_xyz_row, axis=1)

# CV
mean = xyz_clean.mean(axis=1)
std = xyz_clean.std(axis=1)
cv = std / mean
cv = cv.replace([np.inf, -np.inf], 1000).fillna(1000)

# Cuantiles para XYZ
p33 = cv.quantile(0.33)
p67 = cv.quantile(0.67)

xyz_df = pd.DataFrame({'ITEM': xyz_clean.index, 'CV': cv.values})
xyz_df['Clase_XYZ'] = pd.cut(cv, bins=[-1, p33, p67, np.inf], labels=['X', 'Y', 'Z'])
xyz_df['ITEM'] = xyz_df['ITEM'].astype(str).str.strip()

print(f"   ✅ Items clasificados XYZ: {len(xyz_df)}")

# ============================================================================ #
# PARTE 3: COMBINAR ABC-XYZ (✅ CORREGIDO: Conversión a str)
# ============================================================================ #
print("\n🔄 Combinando clasificaciones ABC-XYZ...")

abc_minimal = abc_df[['ITEM', 'Clase_ABC']].copy()
abc_minimal['ITEM'] = abc_minimal['ITEM'].astype(str).str.strip()

# Convertir a string para evitar errores con tipo 'category'
xyz_df['Clase_XYZ'] = xyz_df['Clase_XYZ'].astype(str)

clasificacion_df = abc_minimal.merge(xyz_df[['ITEM', 'Clase_XYZ']], on='ITEM', how='inner')

# Convertir ambas columnas a string antes de concatenar
clasificacion_df['Clase_ABC'] = clasificacion_df['Clase_ABC'].astype(str)
clasificacion_df['Clase_XYZ'] = clasificacion_df['Clase_XYZ'].astype(str)

# Concatenar para crear ABC-XYZ
clasificacion_df['Clasificacion_ABC_XYZ'] = clasificacion_df['Clase_ABC'] + clasificacion_df['Clase_XYZ']

print(f"   ✅ Items con clasificación completa: {len(clasificacion_df)}")

# ============================================================================ #
# PARTE 4: NIVELES DE SERVICIO Y Z-SCORE
# ============================================================================ #
print("\n⚙️ Asignando niveles de servicio por clase...")

niveles_servicio = {
    'AX': 0.95, 'AY': 0.90, 'AZ': 0.85,
    'BX': 0.85, 'BY': 0.80, 'BZ': 0.75,
    'CX': 0.75, 'CY': 0.70, 'CZ': 0.65
}

clasificacion_df['Nivel_Servicio'] = clasificacion_df['Clasificacion_ABC_XYZ'].map(niveles_servicio)
clasificacion_df['Z_Score'] = clasificacion_df['Nivel_Servicio'].apply(
    lambda x: norm.ppf(x) if pd.notna(x) else 1.645
)

print("   📋 Niveles de servicio asignados:")
for clase, nivel in niveles_servicio.items():
    count = len(clasificacion_df[clasificacion_df['Clasificacion_ABC_XYZ'] == clase])
    z = norm.ppf(nivel)
    print(f"      {clase}: {nivel*100:.0f}% (Z={z:.3f}) - {count} items")

# ============================================================================ #
# PARTE 5: STOCK DE SEGURIDAD CON FÓRMULA COMPLETA
# ============================================================================ #
print("\n🛡️ Calculando Stock de Seguridad (modelo completo)...")

historico_df['ITEM'] = historico_df['ITEM'].str.strip()
lead_time_df['ITEM'] = lead_time_df['ITEM'].str.strip()

R_dias = 0.5
R_meses = R_dias / 30
MIN_DATOS = 18

resultados_ss = []

for idx, row in clasificacion_df.iterrows():
    item = row['ITEM']
    Z = row['Z_Score']

    # Buscar datos históricos
    item_hist = historico_df[historico_df['ITEM'] == item]
    if len(item_hist) == 0:
        resultados_ss.append({
            'ITEM': item, 'Stock_Seguridad_Dias': np.nan,
            'Promedio_Ventas_Mensual': np.nan, 'Desviacion_Ventas': np.nan,
            'Lead_Time_Promedio_Dias': np.nan, 'Lead_Time_Desv_Dias': np.nan
        })
        continue

    # Extraer ventas
    ventas = pd.to_numeric(item_hist.iloc[0, 1:], errors='coerce')

    # Imputar negativos
    ventas_imp = ventas.copy()
    negativos = ventas_imp < 0
    for i in negativos[negativos].index:
        pos = list(ventas.index).index(i) if i in ventas.index else None
        if pos is None: continue
        inicio = max(0, pos - 12)
        ventana = ventas.iloc[inicio:pos]
        validos = ventana[(ventana > 0) & (ventana.notna())]
        if len(validos) >= 3:
            estimado = validos.median()
        elif len(validos) > 0:
            estimado = validos.mean()
        else:
            todos = ventas[(ventas > 0) & (ventas.notna())]
            estimado = todos.mean() if len(todos) > 0 else 0
        ventas_imp[i] = max(0, estimado)
    ventas_imp = ventas_imp.clip(lower=0)

    # Últimos 18 meses
    ultimos_18 = ventas_imp[-18:]
    if len(ultimos_18.dropna()) < MIN_DATOS:
        resultados_ss.append({
            'ITEM': item, 'Stock_Seguridad_Dias': np.nan,
            'Promedio_Ventas_Mensual': np.nan, 'Desviacion_Ventas': np.nan,
            'Lead_Time_Promedio_Dias': np.nan, 'Lead_Time_Desv_Dias': np.nan
        })
        continue

    mu_d = ultimos_18.mean()
    sigma_d = ultimos_18.std()
    if pd.isna(sigma_d) or sigma_d == 0:
        sigma_d = 0.1 * mu_d if mu_d > 0 else 0.1

    # Lead Time en DÍAS → convertir a meses
    lt_item = lead_time_df[lead_time_df['ITEM'] == item]
    if len(lt_item) > 0:
        lt_vals = pd.to_numeric(lt_item.iloc[0, 1:], errors='coerce').dropna()
        if len(lt_vals) > 0:
            mu_L_dias = lt_vals.mean()
            sigma_L_dias = lt_vals.std()
        else:
            mu_L_dias, sigma_L_dias = 5.0, 1.0
    else:
        mu_L_dias, sigma_L_dias = 5.0, 1.0

    # Convertir a meses
    mu_L = mu_L_dias / 30
    sigma_L = sigma_L_dias / 30

    # Fórmula completa en unidades
    try:
        SS_unidades = Z * np.sqrt(
            (mu_L + R_meses) * sigma_d**2 +
            (sigma_L**2) * mu_d**2
        )
    except:
        SS_unidades = 0.1

    # Convertir a días
    if mu_d > 0:
        SS_dias = (SS_unidades / mu_d) * 30
        SS_dias = round(SS_dias, 2)
    else:
        SS_dias = 0.0

    resultados_ss.append({
        'ITEM': item,
        'Stock_Seguridad_Dias': SS_dias,
        'Promedio_Ventas_Mensual': round(mu_d, 2),
        'Desviacion_Ventas': round(sigma_d, 2),
        'Lead_Time_Promedio_Dias': round(mu_L_dias, 1),
        'Lead_Time_Desv_Dias': round(sigma_L_dias, 1)
    })

print(f"   ✅ Stock de seguridad calculado para {len(resultados_ss)} items")

# ============================================================================ #
# PARTE 6: CONSOLIDAR Y EXPORTAR
# ============================================================================ #
print("\n📋 Consolidando resultados...")

ss_df = pd.DataFrame(resultados_ss)
resultado_final = clasificacion_df.merge(ss_df, on='ITEM', how='left')

# Unir con datos de valor
resultado_final = resultado_final.merge(
    abc_df[['ITEM', 'Total_Ventas_Anual', 'Precio', 'Valor_Total_Dolares']],
    on='ITEM', how='left'
)

# Reordenar columnas
columnas = [
    'ITEM', 'Clasificacion_ABC_XYZ', 'Clase_ABC', 'Clase_XYZ',
    'Nivel_Servicio', 'Z_Score', 'Stock_Seguridad_Dias',
    'Total_Ventas_Anual', 'Precio', 'Valor_Total_Dolares',
    'Promedio_Ventas_Mensual', 'Desviacion_Ventas',
    'Lead_Time_Promedio_Dias', 'Lead_Time_Desv_Dias'
]

resultado_final = resultado_final[columnas]

# Exportar
resultado_final.to_excel("Analisis_ABC_XYZ_Stock_Seguridad_Completo.xlsx", index=False)

print(f"\n✅ Exportado a: Analisis_ABC_XYZ_Stock_Seguridad_Completo.xlsx")
print(f"\n🎯 Total de ítems: {len(resultado_final)}")

print("\n📊 Stock de Seguridad Promedio por Clase:")
promedio_ss = resultado_final.groupby('Clasificacion_ABC_XYZ')['Stock_Seguridad_Dias'].agg(['mean', 'count']).round(1)
for clase, row in promedio_ss.iterrows():
    print(f"   {clase}: {row['mean']:5.1f} días (n={int(row['count'])})")

print("\n📋 Ejemplo de resultados:")
print(resultado_final[['ITEM', 'Clasificacion_ABC_XYZ', 'Nivel_Servicio', 'Stock_Seguridad_Dias']].head(10).to_string(index=False))

print("="*80)

🚀 INICIANDO ANÁLISIS DE INVENTARIOS - ABC-XYZ + STOCK DE SEGURIDAD MEJORADO

📊 Calculando Clasificación ABC...
   ✅ Items clasificados ABC: 660

📈 Calculando Clasificación XYZ...
   ✅ Items clasificados XYZ: 660

🔄 Combinando clasificaciones ABC-XYZ...
   ✅ Items con clasificación completa: 660

⚙️ Asignando niveles de servicio por clase...
   📋 Niveles de servicio asignados:
      AX: 95% (Z=1.645) - 0 items
      AY: 90% (Z=1.282) - 0 items
      AZ: 85% (Z=1.036) - 0 items
      BX: 85% (Z=1.036) - 0 items
      BY: 80% (Z=0.842) - 0 items
      BZ: 75% (Z=0.674) - 0 items
      CX: 75% (Z=0.674) - 0 items
      CY: 70% (Z=0.524) - 0 items
      CZ: 65% (Z=0.385) - 0 items

🛡️ Calculando Stock de Seguridad (modelo completo)...


/tmp/ipython-input-290555207.py:183: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '24132.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  ventas_imp[i] = max(0, estimado)


   ✅ Stock de seguridad calculado para 660 items

📋 Consolidando resultados...

✅ Exportado a: Analisis_ABC_XYZ_Stock_Seguridad_Completo.xlsx

🎯 Total de ítems: 660

📊 Stock de Seguridad Promedio por Clase:
   Anan:   6.0 días (n=299)
   Bnan:   6.4 días (n=147)
   Cnan:   6.0 días (n=214)

📋 Ejemplo de resultados:
    ITEM Clasificacion_ABC_XYZ  Nivel_Servicio  Stock_Seguridad_Dias
 ITEM 12                  Anan             NaN                  5.83
  ITEM 2                  Anan             NaN                  8.50
  ITEM 1                  Anan             NaN                  4.65
  ITEM 7                  Anan             NaN                  5.03
ITEM 311                  Anan             NaN                  7.78
 ITEM 45                  Anan             NaN                  9.78
ITEM 121                  Anan             NaN                  5.02
ITEM 421                  Anan             NaN                  4.91
 ITEM 63                  Anan             NaN                

In [4]:
import pandas as pd
import numpy as np
import requests
from io import BytesIO
from scipy.stats import norm, mstats
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# URL del archivo Excel en GitHub
url = "https://github.com/santiagonajera/OPTIMIZACI-N-DE-INVENTARIOS-CON-POWER-BI/raw/refs/heads/main/Clases-PowerBi-Inventarios-Datos.xlsx"

print("="*80)
print("🚀 INICIANDO ANÁLISIS DE INVENTARIOS - ABC-XYZ + STOCK DE SEGURIDAD MEJORADO")
print("="*80)

# Descargar archivo desde GitHub
print("📥 Descargando archivo desde GitHub...")
try:
    response = requests.get(url)
    response.raise_for_status()
    excel_data = BytesIO(response.content)
    print("   ✅ Archivo descargado exitosamente")
except Exception as e:
    print(f"   ❌ Error descargando archivo: {e}")
    exit()

# Leer hojas del Excel
try:
    historico_df = pd.read_excel(excel_data, sheet_name='Historico')
    forecast_df = pd.read_excel(excel_data, sheet_name='Forecast')
    precios_df = pd.read_excel(excel_data, sheet_name='Precios-Costos')
    lead_time_df = pd.read_excel(excel_data, sheet_name='LeadTime-Dias')
    print("   ✅ Hojas del Excel cargadas correctamente")
except Exception as e:
    print(f"   ❌ Error leyendo hojas del Excel: {e}")
    exit()

print(f"📊 Datos cargados:")
print(f"   - Histórico: {len(historico_df)} items")
print(f"   - Forecast: {len(forecast_df)} items")
print(f"   - Precios: {len(precios_df)} items")
print(f"   - Lead Time: {len(lead_time_df)} items")

# ============================================================================ #
# PARTE 1: CLASIFICACIÓN ABC
# ============================================================================ #
print("\n📊 Calculando Clasificación ABC...")

# Limpiar nombres de items y columnas
forecast_df['ITEM'] = forecast_df['ITEM'].astype(str).str.strip()
precios_df['ITEM'] = precios_df['ITEM'].astype(str).str.strip()
forecast_df.columns = forecast_df.columns.str.strip()

# Obtener columnas de meses (excluyendo ITEM)
monthly_cols = [col for col in forecast_df.columns if col != 'ITEM']
print(f"   📅 Meses de forecast: {len(monthly_cols)}")

# Calcular ventas anuales totales
forecast_df['Total_Ventas_Anual'] = forecast_df[monthly_cols].sum(axis=1)

# Limpiar precios (remover símbolos $ y comas)
def limpiar_precio(precio):
    """Limpiar formato de precio"""
    if pd.isna(precio):
        return np.nan
    if isinstance(precio, (int, float)):
        return float(precio)
    # Si es string, remover símbolos
    precio_str = str(precio).replace('$', '').replace(',', '').strip()
    try:
        return float(precio_str)
    except:
        return np.nan

precios_df['Precio'] = precios_df['Precio'].apply(limpiar_precio)
precios_df['Costo'] = precios_df['Costo'].apply(limpiar_precio)

# Merge forecast con precios
abc_df = forecast_df[['ITEM', 'Total_Ventas_Anual']].merge(
    precios_df[['ITEM', 'Precio']], on='ITEM', how='inner'
)

# Calcular valor total en dólares
abc_df['Valor_Total_Dolares'] = abc_df['Total_Ventas_Anual'] * abc_df['Precio']

# Ordenar por valor descendente
abc_df = abc_df.sort_values('Valor_Total_Dolares', ascending=False).reset_index(drop=True)

# Calcular porcentaje acumulado
abc_df['Porcentaje_Acumulado'] = (abc_df['Valor_Total_Dolares'].cumsum() /
                                 abc_df['Valor_Total_Dolares'].sum() * 100)

# Clasificar ABC
def clasificar_abc(porcentaje):
    if porcentaje <= 60:
        return 'A'
    elif porcentaje <= 80:
        return 'B'
    else:
        return 'C'

abc_df['Clase_ABC'] = abc_df['Porcentaje_Acumulado'].apply(clasificar_abc)

print(f"   ✅ Items clasificados ABC: {len(abc_df)}")
print(f"      - Clase A: {len(abc_df[abc_df['Clase_ABC'] == 'A'])} items ({len(abc_df[abc_df['Clase_ABC'] == 'A'])/len(abc_df)*100:.1f}%)")
print(f"      - Clase B: {len(abc_df[abc_df['Clase_ABC'] == 'B'])} items ({len(abc_df[abc_df['Clase_ABC'] == 'B'])/len(abc_df)*100:.1f}%)")
print(f"      - Clase C: {len(abc_df[abc_df['Clase_ABC'] == 'C'])} items ({len(abc_df[abc_df['Clase_ABC'] == 'C'])/len(abc_df)*100:.1f}%)")

# ============================================================================ #
# PARTE 2: CLASIFICACIÓN XYZ
# ============================================================================ #
print("\n📈 Calculando Clasificación XYZ...")

# Preparar datos históricos
historico_df['ITEM'] = historico_df['ITEM'].astype(str).str.strip()

# Obtener últimos 18 meses de datos históricos
hist_columns = [col for col in historico_df.columns if col != 'ITEM']
print(f"   📅 Meses históricos disponibles: {len(hist_columns)}")

# Tomar últimos 18 meses
if len(hist_columns) >= 18:
    last_18_months = hist_columns[-18:]
else:
    last_18_months = hist_columns
    print(f"   ⚠️  Solo hay {len(hist_columns)} meses disponibles, usando todos")

# Función para limpiar datos históricos por fila (item)
def clean_xyz_row(row):
    """Limpiar datos de ventas históricas para un item"""
    # Convertir a numérico
    row_numeric = pd.to_numeric(row, errors='coerce')

    # Identificar valores problemáticos (NaN, negativos, ceros)
    problematic = (row_numeric <= 0) | (row_numeric.isna())

    if problematic.any():
        # Calcular mediana de valores válidos
        valid_values = row_numeric[(row_numeric > 0) & (row_numeric.notna())]

        if len(valid_values) >= 3:
            # Si hay suficientes valores válidos, usar la mediana
            fill_value = valid_values.median()
        elif len(valid_values) > 0:
            # Si hay pocos valores válidos, usar el promedio
            fill_value = valid_values.mean()
        else:
            # Si no hay valores válidos, usar 0
            fill_value = 0

        # Reemplazar valores problemáticos
        row_numeric[problematic] = fill_value

    # Asegurar que todos los valores sean no negativos
    row_cleaned = row_numeric.clip(lower=0)

    return row_cleaned

# Procesar datos históricos y calcular CV
print("   🔄 Procesando datos históricos para XYZ...")
xyz_data = historico_df.set_index('ITEM')[last_18_months].copy()

# Aplicar limpieza fila por fila
xyz_clean = xyz_data.apply(clean_xyz_row, axis=1)

# Calcular coeficiente de variación
mean_values = xyz_clean.mean(axis=1)
std_values = xyz_clean.std(axis=1)

# Calcular CV con manejo de casos especiales
cv_values = []
for i in range(len(mean_values)):
    mean_val = mean_values.iloc[i]
    std_val = std_values.iloc[i]

    if mean_val == 0 or pd.isna(mean_val):
        cv = 1000  # CV muy alto para items sin ventas
    elif pd.isna(std_val) or std_val == 0:
        cv = 0  # CV cero para items con variabilidad nula
    else:
        cv = std_val / mean_val

    # Limitar valores extremos
    cv = min(cv, 1000)
    cv_values.append(cv)

# Crear DataFrame con resultados XYZ
xyz_df = pd.DataFrame({
    'ITEM': xyz_clean.index,
    'CV': cv_values
})

# Resetear index y limpiar
xyz_df = xyz_df.reset_index(drop=True)
xyz_df['ITEM'] = xyz_df['ITEM'].astype(str).str.strip()

# Reemplazar infinitos y NaN en CV
xyz_df['CV'] = xyz_df['CV'].replace([np.inf, -np.inf], 1000).fillna(1000)

# Definir cuantiles para clasificación XYZ
cv_series = pd.Series(xyz_df['CV'])
p33 = cv_series.quantile(0.33)
p67 = cv_series.quantile(0.67)

print(f"   📊 Cuantiles CV: P33={p33:.3f}, P67={p67:.3f}")

# Clasificar XYZ usando pd.cut
xyz_df['Clase_XYZ'] = pd.cut(xyz_df['CV'],
                            bins=[-1, p33, p67, np.inf],
                            labels=['X', 'Y', 'Z'])

# Convertir categorías a string para evitar problemas posteriores
xyz_df['Clase_XYZ'] = xyz_df['Clase_XYZ'].astype(str)

print(f"   ✅ Items clasificados XYZ: {len(xyz_df)}")
print(f"      - Clase X: {len(xyz_df[xyz_df['Clase_XYZ'] == 'X'])} items ({len(xyz_df[xyz_df['Clase_XYZ'] == 'X'])/len(xyz_df)*100:.1f}%)")
print(f"      - Clase Y: {len(xyz_df[xyz_df['Clase_XYZ'] == 'Y'])} items ({len(xyz_df[xyz_df['Clase_XYZ'] == 'Y'])/len(xyz_df)*100:.1f}%)")
print(f"      - Clase Z: {len(xyz_df[xyz_df['Clase_XYZ'] == 'Z'])} items ({len(xyz_df[xyz_df['Clase_XYZ'] == 'Z'])/len(xyz_df)*100:.1f}%)")

# ============================================================================ #
# PARTE 3: COMBINAR ABC-XYZ
# ============================================================================ #
print("\n🔄 Combinando clasificaciones ABC-XYZ...")

# Preparar dataframes para merge
abc_minimal = abc_df[['ITEM', 'Clase_ABC']].copy()
abc_minimal['ITEM'] = abc_minimal['ITEM'].astype(str).str.strip()

xyz_minimal = xyz_df[['ITEM', 'Clase_XYZ', 'CV']].copy()
xyz_minimal['ITEM'] = xyz_minimal['ITEM'].astype(str).str.strip()

# Combinar clasificaciones
clasificacion_df = abc_minimal.merge(xyz_minimal, on='ITEM', how='inner')

# Convertir a string antes de concatenar
clasificacion_df['Clase_ABC'] = clasificacion_df['Clase_ABC'].astype(str)
clasificacion_df['Clase_XYZ'] = clasificacion_df['Clase_XYZ'].astype(str)

# Crear clasificación combinada ABC-XYZ
clasificacion_df['Clasificacion_ABC_XYZ'] = (clasificacion_df['Clase_ABC'] +
                                            clasificacion_df['Clase_XYZ'])

print(f"   ✅ Items con clasificación completa: {len(clasificacion_df)}")

# Mostrar distribución por clase
distribucion = clasificacion_df['Clasificacion_ABC_XYZ'].value_counts().sort_index()
print("   📊 Distribución por clase ABC-XYZ:")
for clase, cantidad in distribucion.items():
    print(f"      {clase}: {cantidad} items")

# ============================================================================ #
# PARTE 4: NIVELES DE SERVICIO Y Z-SCORE
# ============================================================================ #
print("\n⚙️ Asignando niveles de servicio por clase...")

# Definir niveles de servicio por clase ABC-XYZ
niveles_servicio = {
    'AX': 0.95, 'AY': 0.90, 'AZ': 0.85,
    'BX': 0.85, 'BY': 0.80, 'BZ': 0.75,
    'CX': 0.75, 'CY': 0.70, 'CZ': 0.65
}

# Asignar nivel de servicio
clasificacion_df['Nivel_Servicio'] = clasificacion_df['Clasificacion_ABC_XYZ'].map(niveles_servicio)

# Calcular Z-score correspondiente
clasificacion_df['Z_Score'] = clasificacion_df['Nivel_Servicio'].apply(
    lambda x: norm.ppf(x) if pd.notna(x) else norm.ppf(0.95)
)

print("   📋 Niveles de servicio asignados:")
for clase, nivel in niveles_servicio.items():
    count = len(clasificacion_df[clasificacion_df['Clasificacion_ABC_XYZ'] == clase])
    if count > 0:
        z = norm.ppf(nivel)
        print(f"      {clase}: {nivel*100:.0f}% (Z={z:.3f}) - {count} items")

# ============================================================================ #
# PARTE 5: STOCK DE SEGURIDAD CON FÓRMULA COMPLETA Y LEAD TIME MEJORADO
# ============================================================================ #
print("\n🛡️ Calculando Stock de Seguridad (modelo completo con lead time mejorado)...")

# Preparar datos de lead time
lead_time_df['ITEM'] = lead_time_df['ITEM'].astype(str).str.strip()

# Función para limpiar lead time con winsorización y mediana
def limpiar_lead_time(lead_times_row, item_name):
    """
    Limpiar datos de lead time aplicando:
    1. Conversión a numérico
    2. Reemplazo de negativos, ceros y errores con mediana
    3. Winsorización para valores extremos (percentiles 5 y 95)
    """
    # Extraer valores de lead time (excluyendo la columna ITEM)
    lt_columns = [col for col in lead_time_df.columns if col != 'ITEM']
    lead_times = lead_times_row[lt_columns]

    # Convertir a numérico
    lead_times_num = pd.to_numeric(lead_times, errors='coerce')

    # Identificar valores problemáticos
    problematic = (lead_times_num <= 0) | (lead_times_num.isna())

    # Calcular mediana de valores válidos para imputación
    valid_values = lead_times_num[(lead_times_num > 0) & (lead_times_num.notna())]

    if len(valid_values) >= 3:
        mediana = valid_values.median()
        # Reemplazar valores problemáticos con la mediana
        lead_times_num[problematic] = mediana

        # Aplicar winsorización (limitar valores extremos a percentiles 5 y 95)
        try:
            # Calcular percentiles
            p5 = valid_values.quantile(0.05)
            p95 = valid_values.quantile(0.95)

            # Aplicar winsorización
            lead_times_winsorized = lead_times_num.clip(lower=p5, upper=p95)
        except:
            # Si falla la winsorización, usar datos sin winsorizar
            lead_times_winsorized = lead_times_num

    elif len(valid_values) > 0:
        # Pocos datos válidos, usar promedio sin winsorización
        promedio = valid_values.mean()
        lead_times_num[problematic] = promedio
        lead_times_winsorized = lead_times_num
    else:
        # No hay datos válidos, usar valor por defecto
        lead_times_winsorized = pd.Series([5.0] * len(lead_times_num), index=lead_times_num.index)

    return lead_times_winsorized

# Parámetros del modelo
R_dias = 0.5  # Tiempo de revisión en días
R_meses = R_dias / 30  # Convertir a meses
MIN_DATOS = 12  # Mínimo de datos requeridos

resultados_ss = []

print(f"   🔄 Procesando {len(clasificacion_df)} items...")

for idx, row in clasificacion_df.iterrows():
    if idx % 100 == 0:
        print(f"      Procesando item {idx+1}/{len(clasificacion_df)}")

    item = row['ITEM']
    z_score = row['Z_Score']

    try:
        # ===== PROCESAR DATOS DE DEMANDA =====
        # Buscar datos históricos del item
        item_hist = historico_df[historico_df['ITEM'] == item]

        if len(item_hist) == 0:
            raise ValueError("Sin datos históricos")

        # Extraer y limpiar ventas históricas (últimos 18 meses)
        ventas_historicas = item_hist[last_18_months].iloc[0]
        ventas_clean = clean_xyz_row(ventas_historicas)

        # Verificar datos suficientes
        datos_validos = (ventas_clean > 0).sum()
        if datos_validos < MIN_DATOS:
            raise ValueError(f"Datos insuficientes ({datos_validos}<{MIN_DATOS})")

        # Calcular estadísticas de demanda mensual
        mu_d = ventas_clean.mean()  # Promedio mensual
        sigma_d = ventas_clean.std()  # Desviación estándar mensual

        # Manejar caso de desviación cero o muy pequeña
        if pd.isna(sigma_d) or sigma_d <= 0:
            sigma_d = 0.1 * mu_d if mu_d > 0 else 0.1

        # ===== PROCESAR DATOS DE LEAD TIME MEJORADO =====
        # Buscar datos de lead time
        item_lt = lead_time_df[lead_time_df['ITEM'] == item]

        if len(item_lt) > 0:
            # Aplicar limpieza avanzada de lead time
            lead_times_clean = limpiar_lead_time(item_lt.iloc[0], item)

            if len(lead_times_clean) > 0:
                mu_L_dias = lead_times_clean.mean()
                sigma_L_dias = lead_times_clean.std()

                # Validar resultados
                if pd.isna(mu_L_dias) or mu_L_dias <= 0:
                    mu_L_dias = 5.0
                if pd.isna(sigma_L_dias) or sigma_L_dias <= 0:
                    sigma_L_dias = 0.1 * mu_L_dias
            else:
                mu_L_dias, sigma_L_dias = 5.0, 1.0
        else:
            # Valores por defecto
            mu_L_dias, sigma_L_dias = 5.0, 1.0

        # Convertir lead time de días a meses
        mu_L = mu_L_dias / 30
        sigma_L = sigma_L_dias / 30

        # ===== APLICAR FÓRMULA COMPLETA DE STOCK DE SEGURIDAD =====
        # SS = Z_α * √[(μL + R)σd² + σL²μd²]
        termino1 = (mu_L + R_meses) * (sigma_d ** 2)
        termino2 = (sigma_L ** 2) * (mu_d ** 2)

        SS_unidades = z_score * np.sqrt(termino1 + termino2)

        # Convertir a días de stock
        if mu_d > 0:
            SS_dias = (SS_unidades / mu_d) * 30
        else:
            SS_dias = 0.0

        # Validar resultado
        if np.isnan(SS_dias) or np.isinf(SS_dias) or SS_dias < 0:
            SS_dias = 0.0

        # Guardar resultados exitosos
        resultados_ss.append({
            'ITEM': item,
            'Stock_Seguridad_Dias': round(SS_dias, 2),
            'Promedio_Ventas_Mensual': round(mu_d, 2),
            'Desviacion_Ventas': round(sigma_d, 2),
            'Lead_Time_Promedio_Dias': round(mu_L_dias, 1),
            'Lead_Time_Desv_Dias': round(sigma_L_dias, 1),
            'Error': None
        })

    except Exception as e:
        # Guardar items con errores
        resultados_ss.append({
            'ITEM': item,
            'Stock_Seguridad_Dias': np.nan,
            'Promedio_Ventas_Mensual': np.nan,
            'Desviacion_Ventas': np.nan,
            'Lead_Time_Promedio_Dias': np.nan,
            'Lead_Time_Desv_Dias': np.nan,
            'Error': str(e)
        })

items_exitosos = len([r for r in resultados_ss if r['Error'] is None])
print(f"   ✅ Stock de seguridad calculado exitosamente para {items_exitosos} items")

# ============================================================================ #
# PARTE 6: CONSOLIDAR Y EXPORTAR
# ============================================================================ #
print("\n📋 Consolidando resultados finales...")

# Crear DataFrame con resultados de stock de seguridad
ss_df = pd.DataFrame(resultados_ss)

# Combinar con clasificación
resultado_final = clasificacion_df.merge(ss_df, on='ITEM', how='left')

# Agregar datos de valor total
resultado_final = resultado_final.merge(
    abc_df[['ITEM', 'Total_Ventas_Anual', 'Precio', 'Valor_Total_Dolares']],
    on='ITEM', how='left'
)

# Reordenar columnas
columnas_finales = [
    'ITEM', 'Clasificacion_ABC_XYZ', 'Clase_ABC', 'Clase_XYZ',
    'Nivel_Servicio', 'Z_Score', 'CV', 'Stock_Seguridad_Dias',
    'Total_Ventas_Anual', 'Precio', 'Valor_Total_Dolares',
    'Promedio_Ventas_Mensual', 'Desviacion_Ventas',
    'Lead_Time_Promedio_Dias', 'Lead_Time_Desv_Dias', 'Error'
]

# Filtrar columnas que existen
columnas_existentes = [col for col in columnas_finales if col in resultado_final.columns]
resultado_final = resultado_final[columnas_existentes]

# Exportar a Excel
nombre_archivo = "Analisis_ABC_XYZ_Stock_Seguridad_Completo.xlsx"
resultado_final.to_excel(nombre_archivo, index=False)

print(f"\n✅ Análisis completado y exportado a: {nombre_archivo}")
print(f"🎯 Total de ítems procesados: {len(resultado_final)}")

# ============================================================================ #
# PARTE 7: RESUMEN Y ESTADÍSTICAS FINALES
# ============================================================================ #
print("\n" + "="*80)
print("📊 RESUMEN FINAL DE RESULTADOS")
print("="*80)

# Items con stock de seguridad calculado
items_con_ss = resultado_final[resultado_final['Stock_Seguridad_Dias'].notna()]
print(f"✅ Items con Stock de Seguridad calculado: {len(items_con_ss)} de {len(resultado_final)} ({len(items_con_ss)/len(resultado_final)*100:.1f}%)")

if len(items_con_ss) > 0:
    print("\n📊 Stock de Seguridad Promedio por Clase ABC-XYZ:")
    resumen_por_clase = items_con_ss.groupby('Clasificacion_ABC_XYZ')['Stock_Seguridad_Dias'].agg(['mean', 'median', 'count', 'std']).round(2)

    for clase, stats in resumen_por_clase.iterrows():
        print(f"   {clase}: μ={stats['mean']:6.1f} días, Med={stats['median']:6.1f}, σ={stats['std']:5.1f} (n={int(stats['count'])})")

    print(f"\n📈 Estadísticas Generales del Stock de Seguridad:")
    print(f"   Promedio general: {items_con_ss['Stock_Seguridad_Dias'].mean():.1f} días")
    print(f"   Mediana: {items_con_ss['Stock_Seguridad_Dias'].median():.1f} días")
    print(f"   Desviación estándar: {items_con_ss['Stock_Seguridad_Dias'].std():.1f} días")
    print(f"   Rango: {items_con_ss['Stock_Seguridad_Dias'].min():.1f} - {items_con_ss['Stock_Seguridad_Dias'].max():.1f} días")

# Mostrar muestra de resultados
print("\n📋 Muestra de Resultados (Top 10 items por valor):")
muestra = resultado_final.nlargest(10, 'Valor_Total_Dolares')[
    ['ITEM', 'Clasificacion_ABC_XYZ', 'Nivel_Servicio', 'Stock_Seguridad_Dias', 'Valor_Total_Dolares']
]
print(muestra.to_string(index=False))

# Contar errores si los hay
errores = resultado_final[resultado_final['Error'].notna()]
if len(errores) > 0:
    print(f"\n⚠️  Items con errores: {len(errores)} ({len(errores)/len(resultado_final)*100:.1f}%)")
    print("   Tipos de errores:")
    tipos_errores = errores['Error'].value_counts()
    for error, cantidad in tipos_errores.items():
        print(f"      {error}: {cantidad} items")

print("\n" + "="*80)
print("🎉 ANÁLISIS ABC-XYZ CON STOCK DE SEGURIDAD COMPLETADO EXITOSAMENTE")
print("🔧 Mejoras aplicadas:")
print("   - Descarga desde repositorio GitHub")
print("   - Limpieza de lead time con winsorización")
print("   - Reemplazo de valores problemáticos con mediana")
print("   - Manejo robusto de errores y casos extremos")
print("="*80)

🚀 INICIANDO ANÁLISIS DE INVENTARIOS - ABC-XYZ + STOCK DE SEGURIDAD MEJORADO
📥 Descargando archivo desde GitHub...
   ✅ Archivo descargado exitosamente
   ✅ Hojas del Excel cargadas correctamente
📊 Datos cargados:
   - Histórico: 660 items
   - Forecast: 660 items
   - Precios: 660 items
   - Lead Time: 660 items

📊 Calculando Clasificación ABC...
   📅 Meses de forecast: 12
   ✅ Items clasificados ABC: 660
      - Clase A: 299 items (45.3%)
      - Clase B: 147 items (22.3%)
      - Clase C: 214 items (32.4%)

📈 Calculando Clasificación XYZ...
   📅 Meses históricos disponibles: 55
   🔄 Procesando datos históricos para XYZ...
   📊 Cuantiles CV: P33=0.151, P67=0.158
   ✅ Items clasificados XYZ: 660
      - Clase X: 218 items (33.0%)
      - Clase Y: 224 items (33.9%)
      - Clase Z: 218 items (33.0%)

🔄 Combinando clasificaciones ABC-XYZ...
   ✅ Items con clasificación completa: 660
   📊 Distribución por clase ABC-XYZ:
      AX: 77 items
      AY: 116 items
      AZ: 106 items
      BX: 